# Query a Fabric Data Agent

Call your **Fabric Data Agent** (Fabric IQ) programmatically. This is the same pattern used by `AirlineOpsContext` in the main app.

```
Your Question (natural language)
        ↓
   Fabric Data Agent  
        ↓
   Translates to SQL/DAX/KQL
        ↓
   Queries OneLake warehouse
        ↓
   Returns formatted answer
```

**Setup:** Replace `PUBLISHED_URL` with your agent's URL from **Fabric → Agent Settings → Published URL**

## 1) Install dependencies

In [20]:
%pip install "openai==1.70.0"
%pip install "synapseml==1.0.5"
%pip install pandas tqdm

^C
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
agent-framework-core 1.0.0b251028 requires aiofiles>=24.1.0, which is not installed.
agent-framework-core 1.0.0b251028 requires openai<2,>=1.99.0, but you have openai 1.70.0 which is incompatible.
mem0ai 1.0.0 requires openai>=1.90.0, but you have openai 1.70.0 which is incompatible.
semantic-kernel 1.28.1 requires pydantic!=2.10.0,!=2.10.1,!=2.10.2,!=2.10.3,<2.12,>=2.0, but you have pydantic 2.12.3 which is incompatible.


  Using cached openai-1.70.0-py3-none-any.whl.metadata (25 kB)
Using cached openai-1.70.0-py3-none-any.whl (599 kB)
  Attempting uninstall: openai
    Found existing installation: openai 1.109.1
    Uninstalling openai-1.109.1:
      Successfully uninstalled openai-1.109.1


## 2) Configure the Fabric client
Uses OpenAI SDK with AAD auth to call your Fabric Data Agent endpoint.

In [1]:
# pip install azure-identity openai==1.70.0

# ---- REQUIRED: paste your Published URL here ----
PUBLISHED_URL = "https://msitapi.fabric.microsoft.com/v1/workspaces/00ae18cb-e789-4d42-be8d-a5b47e524e22/aiskills/1e363010-6005-48da-b370-7906177a760e/aiassistant/openai"

import typing as t
import time, uuid

from azure.identity import InteractiveBrowserCredential
from openai import OpenAI
from openai._models import FinalRequestOptions
from openai._types import Omit
from openai._utils import is_given

# ---------- Dev sign-in ----------
# Opens a browser once; caches token locally
SCOPE = "https://api.fabric.microsoft.com/.default"
# If you see 401/403, swap to:
# SCOPE = "https://analysis.windows.net/powerbi/api/.default"

_cred = InteractiveBrowserCredential()

def _get_bearer() -> str:
    return _cred.get_token(SCOPE).token


class FabricOpenAI(OpenAI):
    """
    OpenAI client wrapper that:
      - Uses your Fabric Data Agent Published URL as base_url
      - Injects AAD Bearer token and correlation id
      - Pins 'api-version' as query param
    """

    def __init__(
        self, api_version: str = "2024-05-01-preview", **kwargs: t.Any
    ) -> None:
        self.api_version = api_version
        default_query = kwargs.pop("default_query", {})
        default_query["api-version"] = self.api_version
        super().__init__(
            api_key="",  # not used
            base_url=PUBLISHED_URL,  # IMPORTANT: your agent endpoint
            default_query=default_query,
            **kwargs,
        )

    def _prepare_options(self, options: FinalRequestOptions) -> None:
        headers: dict[str, str | Omit] = (
            {**options.headers} if is_given(options.headers) else {}
        )
        headers["Authorization"] = f"Bearer {_get_bearer()}"
        headers.setdefault("Accept", "application/json")
        headers.setdefault("ActivityId", str(uuid.uuid4()))
        options.headers = headers
        return super()._prepare_options(options)


client = FabricOpenAI()
print("Client configured. You're signed in with InteractiveBrowserCredential().")

Client configured. You're signed in with InteractiveBrowserCredential().


## 3) Helper function
Ask natural language questions → get data back from Fabric.

In [2]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

def ask_data_agent(
    question: str, poll_interval_sec: int = 2, timeout_sec: int = 300
) -> str:
    """
    Sends a question to the published Fabric Data Agent and returns the text reply.
    Cleans up the thread after completion.
    """
    try:
        # Create "assistant" placeholder (model is ignored by Fabric agent)
        assistant = client.beta.assistants.create(model="not-used")

        # Create a new thread for this Q&A
        thread = client.beta.threads.create()

        try:
            # Post the user message
            client.beta.threads.messages.create(
                thread_id=thread.id,
                role="user",
                content=question,
            )

            # Start a run (the data agent actually does the work)
            run = client.beta.threads.runs.create(
                thread_id=thread.id, assistant_id=assistant.id
            )

            # Poll until terminal state or timeout
            terminal = {"completed", "failed", "cancelled", "requires_action"}
            start = time.time()
            while run.status not in terminal:
                if time.time() - start > timeout_sec:
                    raise TimeoutError(
                        f"Run polling exceeded {timeout_sec}s (last status={run.status})"
                    )
                time.sleep(poll_interval_sec)
                run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

            if run.status != "completed":
                # Get detailed error information
                error_msg = f"❌ Run ended: {run.status}"
                if hasattr(run, 'last_error') and run.last_error:
                    error_code = getattr(run.last_error, 'code', 'unknown')
                    error_message = getattr(run.last_error, 'message', 'No details')
                    error_msg += f"\n   Error Code: {error_code}\n   Error Message: {error_message}"
                    
                    # Add helpful context for common errors
                    if "MWC token" in error_message or "NL2Ontology" in error_message:
                        error_msg += "\n\n💡 This is a Fabric Data Agent configuration issue."
                        error_msg += "\n   The agent endpoint exists but has internal setup problems."
                        error_msg += "\n   Please check:"
                        error_msg += "\n   - Agent is properly published in Fabric workspace"
                        error_msg += "\n   - Data sources are configured and accessible"
                        error_msg += "\n   - Workspace permissions are correct"
                    elif "EntityNotFound" in error_message or "404" in error_message:
                        error_msg += "\n\n💡 The Fabric Data Agent endpoint doesn't exist or was deleted."
                        error_msg += "\n   Please verify the Published URL in your Fabric workspace."
                        
                return error_msg

            # Collect messages in ascending order and concatenate text parts
            msgs = client.beta.threads.messages.list(thread_id=thread.id, order="asc")
            out_chunks = []
            for m in msgs.data:
                if m.role == "assistant":
                    for c in m.content:
                        if getattr(c, "type", None) == "text":
                            out_chunks.append(c.text.value)
            return "\n".join(out_chunks).strip() or "[No text content returned]"

        finally:
            # Always attempt cleanup
            try:
                client.beta.threads.delete(thread_id=thread.id)
            except Exception:
                pass
                
    except Exception as e:
        return f"❌ Exception during agent query: {type(e).__name__}: {str(e)}"


print("✅ Helper ready: call ask_data_agent('your question')")
print("   Deprecation warnings suppressed for cleaner output.")

✅ Helper ready: call ask_data_agent('your question')
   Deprecation warnings suppressed for cleaner output.


## 4) Try it out
Ask questions about your data - Fabric IQ translates to SQL automatically.

In [ ]:
print(ask_data_agent("what are the most repeated routes for SkyBridge Airlines and NorthCoast Airlines in the last year?"))